# Social Network Analysis Example

This notebook demonstrates how to analyze agent interactions using NetworkX.


In [ ]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add parent directory to path
sys.path.insert(0, str(Path().absolute().parent.parent))

from app.sna import SocialNetworkAnalyzer
from app.processors import EventLogProcessor


In [ ]:
# Load event logs
log_file = "../../data/logs/events_20240101_120000.jsonl"

events = []
with open(log_file, "r") as f:
    for line in f:
        if line.strip():
            events.append(json.loads(line))

print(f"Loaded {len(events)} events")


In [ ]:
# Build network graph
sna = SocialNetworkAnalyzer()
graph = sna.build_graph(events)

print(f"Nodes: {len(graph.nodes())}")
print(f"Edges: {len(graph.edges())}")


In [ ]:
# Compute centrality metrics
metrics = sna.compute_metrics(graph)

# Create centrality DataFrame
centrality_df = pd.DataFrame({
    "agent": list(metrics["degree_centrality"].keys()),
    "degree": list(metrics["degree_centrality"].values()),
    "betweenness": list(metrics["betweenness_centrality"].values()),
    "closeness": list(metrics["closeness_centrality"].values()) if "closeness_centrality" in metrics else [0] * len(metrics["degree_centrality"]),
})

print(centrality_df)


In [ ]:
# Visualize network graph
plt.figure(figsize=(12, 8))

pos = nx.spring_layout(graph, k=1, iterations=50)

# Draw nodes with size based on degree centrality
node_sizes = [metrics["degree_centrality"][node] * 3000 for node in graph.nodes()]
nx.draw_networkx_nodes(graph, pos, node_size=node_sizes, node_color="lightblue", alpha=0.7)

# Draw edges with width based on weight
edge_widths = [graph[u][v]["weight"] for u, v in graph.edges()]
nx.draw_networkx_edges(graph, pos, width=edge_widths, alpha=0.5, edge_color="gray")

# Draw labels
nx.draw_networkx_labels(graph, pos, font_size=8)

plt.title("Agent Interaction Network")
plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
# Centrality heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(
    centrality_df.set_index("agent")[["degree", "betweenness", "closeness"]],
    annot=True,
    fmt=".3f",
    cmap="YlOrRd",
)
plt.title("Centrality Metrics Heatmap")
plt.tight_layout()
plt.show()


In [ ]:
# Community detection
communities = sna.detect_communities(graph)

print(f"Number of communities: {communities['num_communities']}")
print(f"Modularity: {communities.get('modularity', 'N/A')}")

# Visualize communities
plt.figure(figsize=(12, 8))

pos = nx.spring_layout(graph, k=1, iterations=50)

# Color nodes by community
node_colors = []
for node in graph.nodes():
    for comm_id, comm_nodes in communities["communities"].items():
        if node in comm_nodes:
            node_colors.append(comm_id)
            break
    else:
        node_colors.append(-1)

nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=500, cmap=plt.cm.Set3, alpha=0.7)
nx.draw_networkx_edges(graph, pos, alpha=0.3, edge_color="gray")
nx.draw_networkx_labels(graph, pos, font_size=8)

plt.title("Community Detection")
plt.axis("off")
plt.tight_layout()
plt.show()
